<a href="https://colab.research.google.com/github/TimothyJan/RNN_Text_Generation/blob/main/Project_4_RNN_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Spring 2022 - CPSC 585-section-13883
Project 4 - Recurrent Neural Networks Text Generation
Sean Javiya
Timothy Jan
Timothy Kheang

In Project 4, we use an RNN-based language model to generate text for a creative application.

Goals for this project are:
<ul>
  <li>Reading about how RNNs can be used to generate text, and examining several different applications after training on different text corpora.</li>
  <li>Using a multi-layer RNN to train and sample from a character-level language model.</li>
  <li>Adapting and reusing published model code.</li>
  <li>Having some fun.</li>
</ul>

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import urllib.request, json 

Download the Elder Scroll's or Shakespearean dataset and read the data.

In [ ]:
#'''imported the elder scrolls json file'''
#with urllib.request.urlopen("https://raw.githubusercontent.com/hmi-utwente/video-game-text-corpora/master/The%20Elder%20Scrolls/data/imperial_library_20200626.json") as url:
#    data_dict = json.loads(url.read().decode())
#text=""
#for key in data_dict:
#  text += data_dict[key]['description'] +' ' + data_dict[key]['title'] + ' ' + data_dict[key]['text']
#text[:100]

#'''imported the aesops json file'''
#with urllib.request.urlopen("https://raw.githubusercontent.com/itayniv/aesop-fables-stories/master/public/aesopFables.json") as url:
#    data_dict = json.loads(url.read().decode())
#text=""
#for key, value in data_dict.items():
#  for story in value:
#    temp = dict(story)
#    text += str(temp["story"]) + " "
#print(text[:100])

# '''imported the Harry Potter text files'''
text=""
with urllib.request.urlopen("https://raw.githubusercontent.com/ErikaJacobs/Harry-Potter-Text-Mining/master/Book%20Text/HPBook1.txt") as url:
    text += url.read().decode()
with urllib.request.urlopen("https://raw.githubusercontent.com/ErikaJacobs/Harry-Potter-Text-Mining/master/Book%20Text/HPBook2.txt") as url:
    text += url.read().decode()
with urllib.request.urlopen("https://raw.githubusercontent.com/ErikaJacobs/Harry-Potter-Text-Mining/master/Book%20Text/HPBook3.txt") as url:
    text += url.read().decode()
with urllib.request.urlopen("https://raw.githubusercontent.com/ErikaJacobs/Harry-Potter-Text-Mining/master/Book%20Text/HPBook4.txt") as url:
    text += url.read().decode()
with urllib.request.urlopen("https://raw.githubusercontent.com/ErikaJacobs/Harry-Potter-Text-Mining/master/Book%20Text/HPBook5.txt") as url:
    text += url.read().decode()
with urllib.request.urlopen("https://raw.githubusercontent.com/ErikaJacobs/Harry-Potter-Text-Mining/master/Book%20Text/HPBook6.txt") as url:
    text += url.read().decode()
with urllib.request.urlopen("https://raw.githubusercontent.com/ErikaJacobs/Harry-Potter-Text-Mining/master/Book%20Text/HPBook7.txt") as url:
    text += url.read().decode()

# '''Shakespearen dataset'''
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# # Read, then decode for py2 compat.
# text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# # length of text is the number of characters in it
# print(f'Length of text: {len(text)} characters')
# print(text[:100])

Read the Data

In [ ]:
# unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

104 unique characters


<h1>Process the text</h1>
<h2>Vectorize the text.</h2>
Before training, you need to convert the strings to a numerical representation.
The tf.keras.layers.StringLookup layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Create the tf.keras.layers.StringLookup layer. It onverts from tokens to character IDs:

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[66, 67, 68, 69, 70, 71, 72], [89, 90, 91]]>

This layer recovers the characters from the vectors of IDs, and returns them as a tf.RaggedTensor of characters:

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Use tf.strings.reduce_join to join the characters back into strings.

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


<h2>The Prediction Task</h2>
Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

<h2>Create training examples and targets</h2>
Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(6325064,), dtype=int64, numpy=array([ 5, 53, 70, ..., 33, 24,  2])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

"
T
e
x
t
"
@
"
C
h


The batch method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'"' b'T' b'e' b'x' b't' b'"' b'@' b'"' b'C' b'h' b'a' b'p' b't' b'e'
 b'r' b'"' b'@' b'"' b'B' b'o' b'o' b'k' b'"' b'\n' b'"' b'T' b'H' b'E'
 b' ' b'B' b'O' b'Y' b' ' b'W' b'H' b'O' b' ' b'L' b'I' b'V' b'E' b'D'
 b' ' b' ' b'M' b'r' b'.' b' ' b'a' b'n' b'd' b' ' b'M' b'r' b's' b'.'
 b' ' b'D' b'u' b'r' b's' b'l' b'e' b'y' b',' b' ' b'o' b'f' b' ' b'n'
 b'u' b'm' b'b' b'e' b'r' b' ' b'f' b'o' b'u' b'r' b',' b' ' b'P' b'r'
 b'i' b'v' b'e' b't' b' ' b'D' b'r' b'i' b'v' b'e' b',' b' ' b'w' b'e'
 b'r' b'e' b' '], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'"Text"@"Chapter"@"Book"\n"THE BOY WHO LIVED  Mr. and Mrs. Dursley, of number four, Privet Drive, were '
b"proud to say that they were perfectly normal, thank you very much. They were the last people you'd ex"
b"pect to be involved in anything strange or mysterious, because they just didn't hold with such nonsen"
b'se.  Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy '
b'man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blond'


For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'"Text"@"Chapter"@"Book"\n"THE BOY WHO LIVED  Mr. and Mrs. Dursley, of number four, Privet Drive, were'
Target: b'Text"@"Chapter"@"Book"\n"THE BOY WHO LIVED  Mr. and Mrs. Dursley, of number four, Privet Drive, were '


<h2>Create training batches</h2>
You used tf.data to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

<h1>Build The Model</h1>
This section defines the model as a keras.Model subclass (For details see Making new Layers and Models via subclassing).

This model has three layers:
<ul>
  <li>tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;</li>
  <li>tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)</li>
  <li>tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.</li>
</ul>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

<h1>Try the model</h1>
Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 105) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  26880     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  107625    
                                                                 
Total params: 4,072,809
Trainable params: 4,072,809
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.
using the first example in the batch, this gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([ 67,  48,  29,  86,  18,   5,  26,  84,  40,  77,  10,   5,  69,
        58,  60,  70,  90,  67,  96,  60,  23,  39,  58,  33,  31,  27,
        55,  65,  49,  52,  67,  75,  38,  97,  97,  25,  72,   7,  48,
        31,  96,  68,  59,  96,  22,  95,  86,  61,  49,  65,   2,  93,
        66, 101,  25,  98, 102,  76,  20,  19,  80,  16,   0,   7,  45,
        66,  39,  95,  61,  18,  85,  60, 103,  43,  21,  70,  80,  90,
        89,  22,  30,  44,  49,  85,   0,  57,   5,  71,  58,  40,  35,
        73,  96,  34,  80,  80,  94,   9,  97,  64])

Decoded text predicted by untrained model:


In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'arry\'s legs began to jerk around out of his control in a kind of quickstep.  \\"Stop! Stop!\\" screame'

Next Char Predictions:
 b'bO<u1"9sGl("dY[eyb\xc2\xa6[6FY@>:V`PSbjE\xc2\xa8\xc2\xa88g%O>\xc2\xa6cZ\xc2\xa65\xc2\xa0u\\P`\n}a\xe2\x80\x998\xe2\x80\x93\xe2\x80\x9ck32o/[UNK]%LaF\xc2\xa0\\1t[\xe2\x80\x9dJ4eoyx5=KPt[UNK]X"fYGBh\xc2\xa6Aoo~\'\xc2\xa8_'


<h1>Train the model</h1>
Given the previous RNN state, and the input this time step, predict the class of the next character.
<h2>Attach an optimizer, and a loss function</h2>
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, you need to set the from_logits flag.

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 105)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.6527653, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

104.874596

Configure the training procedure using the tf.keras.Model.compile method. Use tf.keras.optimizers.Adam with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

<h2>Configure Checkpoints</h2>
Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

<h2>Execute the training</h2>
To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
#EPOCHS = 50
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
978/978 [==============================] - 134s 129ms/step - loss: 1.7618
Epoch 2/20
345/978 [=========>....................] - ETA: 1:22 - loss: 1.2532

<h1>Generate text</h1>
The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.<br>
Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Character_Name:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

The easiest thing you can do to improve the results is to train it for longer (try EPOCHS = 30).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text faster the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['name:', 'name:', 'name:', 'name:', 'name:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

<h2>Export the generator</h2>
This single-step model can easily be saved and restored, allowing you to use it anywhere a tf.saved_model is accepted.


In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['name:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io

path = keras.utils.get_file(
    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()